# **Limpieza de Datos**

Identificar y manejar valores nulos, duplicados, inconsistencias o posibles outliers.

# 1.- **Importar librerias**

In [4]:
# Importar librerías estándar
import sys
import os

# 2. Configurar la ruta para que Python encuentre el código en 'src'. Esto añade la carpeta raíz del proyecto al sys.path
ruta_raiz_proyecto = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(ruta_raiz_proyecto)

# 3. Importar las funciones directamente.
from src.cargar_analisis import cargar_dataframe, crear_listas_variables
from src.limpieza import limpiar_y_detectar_atipicos, eliminar_atipicos, guardar_dataframe
from src.carga_dvc import run, save_csv_and_push

print("Funciones importadas exitosamente desde src/")

Funciones importadas exitosamente desde src/


# **2.- Importar datos**

In [5]:
# Ruta de la base de datos
path_data = 'd:/OneDrive/Escritorio/Maestria IA/Trimestre 4/MLOps/Git_Local/ObesityEstimation53/data/obesity_estimation_cleaned.csv'

# Ejecutar función para cargar datos
df = cargar_dataframe(path_data)

# Ejecutar la función para generar listas con las variables numericas, categoricas y objetivo
variables_numericas, variables_categoricas, variable_objetivo = crear_listas_variables(to_lower=True)

Archivo CSV cargado exitosamente desde: d:/OneDrive/Escritorio/Maestria IA/Trimestre 4/MLOps/Git_Local/ObesityEstimation53/data/obesity_estimation_cleaned.csv


# **3.- Limpieza de datos**

In [6]:
# Paso 1: Crear una copia limpia del DataFrame
df_limpio = df.copy()

# Estandarizar nombres de columnas
df_limpio.columns = df_limpio.columns.str.strip().str.replace(' ', '_').str.lower()

# Eliminar duplicados
df_limpio = df_limpio.drop_duplicates()

print("DataFrame limpio creado con éxito")

DataFrame limpio creado con éxito


In [7]:
# Paso 2: Eliminar los atípicos del dataframe ya limpio
df_final = eliminar_atipicos(df_limpio, age_range=(5, 50), height_max=2.5, ncp_max=10.0, iqr_factor=1.5)
df_final.head()

---

## **Proceso de Eliminación de Atípicos**

---

1. Aplicando reglas de negocio para 'age', 'height' y 'ncp'...
   - Se eliminaron 0 filas con reglas de negocio.

2. Aplicando método IQR para el resto de variables numéricas...
   - Se eliminaron 0 filas adicionales con el método IQR.


### Reporte Final de Eliminación

Filas iniciales: 1,909
Filas finales:   1,909
Total de filas eliminadas: 0 (0.00%)


---


Proceso de eliminación de atípicos finalizado.


,gender,age,height,weight,family_history_with_overweight,favc,fcvc,ncp,caec,smoke,ch2o,scc,faf,tue,calc,mtrans,nobeyesdad
0,female,21.0,1.62,64.0,yes,no,2.0,3.0,sometimes,no,2.0,no,0.0,1.0,no,public_transportation,1
1,female,21.0,1.52,56.0,yes,no,3.0,3.0,sometimes,yes,3.0,yes,3.0,0.0,sometimes,public_transportation,1
2,male,23.0,1.80,77.0,yes,no,2.0,3.0,sometimes,no,2.0,no,2.0,1.0,frequently,public_transportation,1
3,male,27.0,1.80,87.0,no,no,3.0,3.0,sometimes,no,2.0,no,2.0,0.0,frequently,walking,2
4,male,22.0,1.78,89.8,no,no,2.0,1.0,sometimes,no,2.0,no,0.0,0.0,sometimes,public_transportation,3


# **4.- Guardar los Resultados**

In [8]:
# Llamar a la función para guardar resultados
import os
nombre_archivo = 'obesity_estimation_cleaned.csv'
ruta_destino = 'd:/OneDrive/Escritorio/Maestria IA/Trimestre 4/MLOps/Git_Local/ObesityEstimation53/data/'

# Crear la carpeta si no existe
os.makedirs(ruta_destino, exist_ok=True)

output_path = os.path.join(ruta_destino, nombre_archivo)
guardado_exitoso = False
try:
    df_final.to_csv(output_path, index=False)
    guardado_exitoso = True
    print(f"\nArchivo guardado exitosamente en: {output_path}")
except Exception as e:
    print(f"\nError al guardar el archivo: {e}")

# Resultado
if guardado_exitoso:
    print("\nProceso de guardado finalizado con éxito.")
else:
    print("\nProceso de guardado fallido. Revisar la ruta destino.")


Archivo guardado exitosamente en: d:/OneDrive/Escritorio/Maestria IA/Trimestre 4/MLOps/Git_Local/ObesityEstimation53/data/obesity_estimation_cleaned.csv

Proceso de guardado finalizado con éxito.


# **5.- Guardar los Resultados en DVC**

In [9]:
# Guardar dataset limpio y actualizar DVC/Git
try:
    # Ruta explícita al proyecto
    project_root = r'd:\OneDrive\Escritorio\Maestria IA\Trimestre 4\MLOps\Git_Local\ObesityEstimation53'
    
    print(f"Cambiando al directorio: {project_root}")
    os.chdir(project_root)
    
    # Guardar el CSV
    output_path = os.path.join(project_root, "data", "obesity_estimation_cleaned.csv")
    df_final.to_csv(output_path, index=False)
    print(f"CSV guardado localmente en: {output_path}")
    
    # Comandos DVC y Git
    from src.carga_dvc import run
    
    # Configurar Git
    try:
        run('git config user.name "MLOps Team53"')
        run('git config user.email "team53@mlops.com"')
    except:
        pass
        
    # Asegurar que estamos en la rama correcta
    try:
        run('git checkout dev2')
    except:
        run('git checkout -b dev2')
    
    # Actualizar .gitignore principal
    with open('.gitignore', 'w') as f:
        f.write("""
/data/**
!/data/*.dvc
        """)
    
    # Inicializar DVC si es necesario
    if not os.path.exists('.dvc'):
        run('dvc init')
    
    # Configurar DVC para usar GitHub
    try:
        run('dvc remote remove storage')
    except:
        pass
    
    # Usar GitHub como almacenamiento DVC
    run('dvc remote add -f -d storage https://github.com/JCGarcesDC/ObesityEstimation53.git')
    
    # Añadir el archivo a DVC (forzar si es necesario)
    try:
        run('dvc remove data/obesity_estimation_cleaned.csv.dvc')  # Eliminar si existe
    except:
        pass
        
    run('dvc add -f data/obesity_estimation_cleaned.csv')
    print("Archivo añadido a DVC.")
    
    # Stage changes
    run('git add -f .gitignore')
    run('git add -f data/obesity_estimation_cleaned.csv.dvc')
    run('git add -f .dvc/config')
    
    # Commit
    run('git commit -m "Update cleaned dataset and DVC config"')
    print("Cambios commiteados en Git.")
    
    print("\nPara completar el proceso, necesitas ejecutar manualmente los siguientes comandos:")
    print("\n1. Hacer push a GitHub:")
    print('git push origin dev2')
    print("\n2. Configurar el token de GitHub (reemplaza YOUR_TOKEN con tu token personal):")
    print('git remote set-url origin https://YOUR_TOKEN@github.com/JCGarcesDC/ObesityEstimation53.git')
    print("\n3. Push a DVC:")
    print('dvc push')
    
except Exception as e:
    print(f"Error: {str(e)}")
    print("\nEstado actual:")
    print(f"Directorio actual: {os.getcwd()}")
    if os.path.exists('.dvc'):
        print("El directorio .dvc existe")
        print("Contenido de .dvc:")
        print(os.listdir('.dvc'))
        print("\nContenido de .dvc/config:")
        with open('.dvc/config', 'r') as f:
            print(f.read())

Cambiando al directorio: d:\OneDrive\Escritorio\Maestria IA\Trimestre 4\MLOps\Git_Local\ObesityEstimation53
CSV guardado localmente en: d:\OneDrive\Escritorio\Maestria IA\Trimestre 4\MLOps\Git_Local\ObesityEstimation53\data\obesity_estimation_cleaned.csv
Archivo añadido a DVC.
Archivo añadido a DVC.
On branch dev2
Your branch is up to date with 'origin/dev2'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   artefactos/mejores_modelos.csv
	modified:   artefactos/pipelines_optimizados.joblib
	modified:   notebooks/1. Data manipulation and preparation/1.2_data_cleaning.ipynb
	modified:   notebooks/1. Data manipulation and preparation/1.3_exploratory_analysis_eda.ipynb
	modified:   notebooks/2. Preprocessing and Feature Engineering/2.1_feature_engineering.ipynb
	modified:   notebooks/2. Preprocessing and Feature Engineering/2.2_preprocessing_pipelines.ipynb
	modi